In [1]:
from mistralai import Mistral
import os

from common.keys import MISTRAL_API_KEY_PAYANTE
from common.path import COMMERCIAL_ONE_DRIVE_PATH, rapatrie_file

client = Mistral(api_key=MISTRAL_API_KEY_PAYANTE)

CHIEN_QUI_FUME_PATH = (
    COMMERCIAL_ONE_DRIVE_PATH
    / "2 - DOSSIERS à l'ETUDE"
    / "CHIEN QUI FUME (Le) - 75001 PARIS - 33 Rue du PONT-NEUF"
)

pdf_path = (
    CHIEN_QUI_FUME_PATH / "3. DOCUMENTATION FINANCIÈRE" / "2022 - GALLA - BILAN.pdf"
)
pdf_file_path = rapatrie_file(pdf_path)



logfile available C:\Users\lvolat\Documents\cocoAI\cocoAI\ipykernel_launcher.log


cocoAI - INFO - 2022 - GALLA - BILAN.pdf already exists in 3._DOCUMENTATION_FINANCIRE


In [2]:

uploaded_pdf = client.files.upload(
    file={
        "file_name": pdf_file_path.name,
        "content": open(pdf_file_path, "rb"),
    },
    purpose="ocr"
)  

In [3]:
client.files.retrieve(file_id=uploaded_pdf.id)

RetrieveFileOut(id='446503b2-b83e-40fc-9fd7-754fe052af78', object='file', size_bytes=3220883, created_at=1742567625, filename='2022_-_GALLA_-_BILAN.pdf', purpose='ocr', sample_type='ocr_input', source='upload', deleted=False, num_lines=None)

In [4]:
signed_url = client.files.get_signed_url(file_id=uploaded_pdf.id)

In [ ]:
# from mistralai import Mistral
# client = Mistral(api_key=MISTRAL_API_KEY_PAYANTE)


In [ ]:
ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "document_url",
        "document_url": signed_url.url,
    }
)

In [ ]:
# Import required libraries
import json

from IPython.display import Markdown, display
from mistralai import DocumentURLChunk, Mistral
from mistralai.models import OCRResponse
from common.path import COMMERCIAL_ONE_DRIVE_PATH, rapatrie_file


api_key = MISTRAL_API_KEY_PAYANTE  # Replace with your API key
client = Mistral(api_key=api_key)



# Upload PDF file to Mistral's OCR service
uploaded_file = client.files.upload(
    file={
        "file_name": pdf_file_path.stem,
        "content": pdf_file_path.read_bytes(),
    },
    purpose="ocr",
)

# Get URL for the uploaded file
signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)

# Process PDF with OCR, including embedded images
pdf_response = client.ocr.process(
    document=DocumentURLChunk(document_url=signed_url.url),
    model="mistral-ocr-latest",
    include_image_base64=True,
)

# Convert response to JSON format
response_dict = json.loads(pdf_response.model_dump_json())

print(json.dumps(response_dict, indent=4)[0:1000])  # check the first 1000 characters



In [ ]:

def replace_images_in_markdown(markdown_str: str, images_dict: dict) -> str:
    """
    Replace image placeholders in markdown with base64-encoded images.

    Args:
        markdown_str: Markdown text containing image placeholders
        images_dict: Dictionary mapping image IDs to base64 strings

    Returns:
        Markdown text with images replaced by base64 data
    """
    for img_name, base64_str in images_dict.items():
        markdown_str = markdown_str.replace(
            f"![{img_name}]({img_name})", f"![{img_name}]({base64_str})"
        )
    return markdown_str


def get_combined_markdown(ocr_response: OCRResponse) -> str:
    """
    Combine OCR text and images into a single markdown document.

    Args:
        ocr_response: Response from OCR processing containing text and images

    Returns:
        Combined markdown string with embedded images
    """
    markdowns: list[str] = []
    # Extract images from page
    for page in ocr_response.pages:
        image_data = {}
        for img in page.images:
            image_data[img.id] = img.image_base64
        # Replace image placeholders with actual images
        markdowns.append(replace_images_in_markdown(page.markdown, image_data))

    return "\n\n".join(markdowns)


# Display combined markdowns and images
# display(Markdown(get_combined_markdown(pdf_response)))



In [ ]:
import os
from mistralai import Mistral

# Retrieve the API key from environment variables
api_key = MISTRAL_API_KEY_PAYANTE

# Specify model
model = "mistral-small-latest"

# Initialize the Mistral client
client = Mistral(api_key=api_key)

# Define the messages for the chat
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "quel est le montant des immoblisations incorporelles pour l'exercice 2021?"
                # "text": "quel est le chiffre d'affaires en 2022 ?"
            },
            {
                "type": "document_url",
                "document_url": signed_url.url
            }
        ]
    }
]

# Get the chat response
chat_response = client.chat.complete(
    model=model,
    messages=messages
)

# Print the content of the response
print(chat_response.choices[0].message.content)